In [ ]:
%pip install networkx gravis nbformat nbconvert

In [ ]:
import ast
import nbformat
from nbconvert import PythonExporter
from networkx import MultiDiGraph, relabel_nodes
import gravis as gv

def parse_ipynb_to_ast(file_path):
    # Read the ipynb file
    with open(file_path, 'r') as f:
        nb = nbformat.read(f, as_version=4)

    # Convert the notebook to Python code
    exporter = PythonExporter()
    (python_code, _) = exporter.from_notebook_node(nb)

    # Parse the Python code to AST
    parsed_ast = ast.parse(python_code)

    return parsed_ast



def parse_ast_to_graph(ast_node):
    graph = MultiDiGraph()

    # Traverse the AST and add edges to the graph 
    def traverse(node, parent=None):
        for child in ast.iter_child_nodes(node):
            graph.add_edge(node, child, type='parent-child')
            traverse(child, node)
            # Add sibling relationships
            for sibling in ast.iter_child_nodes(node):
                if sibling is not child:
                    type = node.__class__.__name__ + '-' + sibling.__class__.__name__
                    graph.add_edge(node, sibling, type=type)

    traverse(ast_node)

    return graph

def rename_node(node):
    if isinstance(node, ast.AST):
        return node.__class__.__name__
    return node

def prep_graph_for_gravis(graph):
    # Rename nodes to their class names
    graph = graph.copy()
    graph = relabel_nodes(graph, rename_node)

    return graph



gv.d3(prep_graph_for_gravis(parse_ast_to_graph(parse_ipynb_to_ast('runme.ipynb'))))

In [ ]:
import ast
import nbformat
from nbconvert import PythonExporter
from networkx import MultiDiGraph, relabel_nodes
import gravis as gv

def parse_ipynb_to_ast(file_path):
    # Read the ipynb file
    with open(file_path, 'r') as f:
        nb = nbformat.read(f, as_version=4)

    # Convert the notebook to Python code
    exporter = PythonExporter()
    (python_code, _) = exporter.from_notebook_node(nb)

    # Parse the Python code to AST
    parsed_ast = ast.parse(python_code)

    return parsed_ast

def parse_ast_to_graph(ast_node):
    graph = MultiDiGraph()

    # Traverse the AST and add edges to the graph
    def traverse(node, parent=None):
        for child in ast.iter_child_nodes(node):
            graph.add_edge(node, child, type='parent-child')
            traverse(child, node)
            # Add variable interactions
            if isinstance(child, ast.Assign):
                for target in child.targets:
                    if isinstance(target, ast.Name):
                        graph.add_edge(child, target, type='assignment')
            elif isinstance(child, ast.Name):
                graph.add_edge(parent, child, type='usage')

    traverse(ast_node)

    return graph

def rename_node(node):
    if isinstance(node, ast.AST):
        if isinstance(node, ast.Name):
            return f"{node.id}"
        return node.__class__.__name__
    return node

def prep_graph_for_gravis(graph):
    # Rename nodes to their class names
    graph = graph.copy()
    graph = relabel_nodes(graph, rename_node)
    return graph

gv.d3(prep_graph_for_gravis(parse_ast_to_graph(parse_ipynb_to_ast('runme.ipynb'))))

In [ ]:
gv.three(prep_graph_for_gravis(parse_ast_to_graph(parse_ipynb_to_ast('runme.ipynb'))))

In [ ]:
gv.vis(prep_graph_for_gravis(parse_ast_to_graph(parse_ipynb_to_ast('runme.ipynb'))))